In [1]:
from __future__ import division

import re
import pandas as pd
from nltk.corpus import stopwords
from nltk import FreqDist
from clean_str import clean_str

langs = {'en':'4chan/messages-b.csv', 'ru':'2ch/messages-b.csv', 
    'es':'nido/messages-b.csv', 'de':'krautchan/messages-b.csv',
    'mdk':'mdk/messages.csv'}
obscene_voc = set(['slut','fags','fag','faggots','fucking', 'fucked','shit','rape','faggot','kill','bitch','dick'
               'asshole','shit','piss', 'fuck', 'cunt', 'cocksucker', 'motherfucker', 'tits', 'die', 'arse',
                'crap', 'damn', 'goddman', 'ass', 'nigger', 'dick', 'pussy', 'asshole', 'niggers', 'dumb', 'cock', 'retarded',
                'retard', 'stupid', 'cum', 'balls', 'autism', 'hate', 'retards', 'newfags', 'idiot', 'pussies', 'dumbass',
                'autistic', 'fap', 'jew', 'cocks', 'shitty', 'assholeness', 'nigga', 'bitches', 'niggerise', 'leftistsfucking',
                'rightdumb',
    
               'fresse','mulle', 'mullen','ficken', 'fick', 'arsch', 'fotze', u'miststück', 'schlampe', 'schwul',
               'verpiss', 'verpissen', 'verpisst', 'fickt', 'neger', u'scheiß', u'scheiße', 'schwein', 'pene',
               'titten', 'maul', 'dreck',
               u'охуевший',u'охуел',u'охуенно',u'пиздёж',u'пиздеж',u'блять',u'блядь',u'хуевой',u'хуевый',u'хуево',
               u'социоблядков',u'социблядок',u'уебища',u'уебище',u'ебаной',u'ебаных',u'ебаный',u'ебаная',u'проебал',
               u'ебанулись',u'ебанулся',u'ебанутый',u'поебаться',u'поебался',u'хуйню',u'подъебать',u'проебано',
               u'заебать',u'заебало',u'хуйло',u'хуйня',u'пиздец',u'пизда',u'хуй',u'сука', u'ебал', u'ебут', u'ебу',
               u'жопа', u'петушара', u'говно', u"нахуй", u"быдло", u"дебил", u"бля", u'нихуя', u'даун', u'похуй',
               u'говна', u"тупой", u"сучек"
               u'пидор', u'ебать', u'хули', u'пидора', u'пидоры', u'соси', u'обоссал', u'долбоеб', u'нахуя',
               u'быдла', u'жрут', u'хуи', u'хуйней', u'ебанутые', u'уебок', u'быдлу', u'ебало', u'идиот', u'хуесос', u'охуеть',
               u'шлюха', u'суки', u'уебывай', u'пидорас', u'дохуя', u'заебал', u'пиздит', u'дауну', u'ссать', u'шкуры', u'тупее',
               u'пиздос', u'бляди', u'жопой', u'ебeт', u'пидоров', u'пиздеть', u'моча', u'дерьмо', u'жрать', u'пизду', u'уeбищной',
               u'обосрался', u'съеби', u'ебучие', u'ебучий', u'драть', u'ссаный', u'быдлан', u'хуйни', u'дрочи', u'заебали',
               u'шлюхой', u'тупое', u'пидорство', u'тупая', u'шлюху', u'конченые', u'хачи', u'блеать', u'мочи', u'мрази', u'хуя',
               u'быдлом', u'ебаные', u'свинья', u'хохла', u'свиньи', u'пидорах', u'анус', u'срач', u'пердак', u'ебали', u'ебалу',
               u'поехавший', u'хуле', u'урод', u'пиздабол', u'лошков', u'бла', u'ебнул', u'дебиловиди', u'соснули', u'долбаeбам',
               u'заебок', u'имбецил', u'еблан', u'соснуло', u'мусарам', u'отпиздили', u'рассию', u'ахуе', u'ахуеть', u'долбаeб',
               u'хуесосила', u'проебана', u'анально', u'дегенерат', u'днище',
               u'мент', u'гандон', u'епать', u'обосрыши', u'саснуть', u'пиzдюк', u'пездюк', u'задротка', u'ублюдос', u'позорься',
               u'колхозник', u'педики', u'шкура',u'говорятпидарам', u'пилоток', u'захлебнулась',
               u'дибилы', u'похер', u'охуели', u'обоссаных',
               'mierda', 'chucha', 'fea', 'hueonesmina', 'raja', 'chupa', 'conchetumare',
               'culo', 'feminazis', 'maricas', 'gorda', 'negros', 'culiao', 'culiar', 'ahueonao', 'hueonao', 'reculiao',
               'cabros', 'culeas', 'culeados', 'puta', 'injuria', 'forro', 'boludo', 
    
              ])
obscene_roots = (u'хуй', u"хуе", u"хуя", u"муда", u"муди", u'дегенерат', u'наеб', u'поеб', u'лох', u"лош", u"пизд", 
                 u'пидор', u'еба', u'ебе', u"выеб", u"спизд", u"попизд", "fuck", "dick", "fick", u"быдл", u"уеб",
                 u'петушар', u'даун', u'мент', u'пидр', u'ебы', u'сперм',
                 u"пидар", u"наху", u"долбо", u"долба", u"ебу", u"схуя", u"говн", u"отпизд", u"хуев", u"охуен", u"хохл", u"сук",
                 u"дроч", u"сьеб", u"хую", u"шлюх", u"соса", "dick", "fag", "cunt", "nig", "shit", "ass", "arse",
                 "auti", "retar", "cock", 'verpiss', 'verpissen', 'verpisst', 'fickt', 'neger', u'scheiß', 'schwein', 'pene',
                'titt', 'maul', 'dreck', 'pendej', 'mierd', 'chuch', 'raj', 'chup', 'cul', 'maric', 'huen', 'cabr',
                'ching', 'pinch', 'joder', 'joda', 'huevo', 'gilipolla', 'malparida', 'piruja', 'zopupla', 'zorra',
                'puta', 'pajer', 'joto', 'verga')

In [2]:
def analyse(lang):
    print(lang)
    f = open('annotate_' + lang + '_short')
    annotated = f.readlines()
    f.close()
    annotated = [a.strip().split('\t') for a in annotated]
    print('Annotated length: {}'.format(len(annotated)))
    ratio = 1
    if len(annotated) < 2000:
        ratio = 2000/len(annotated)
    print(ratio)
    message_numbers = set([a[1] for a in annotated])
    positive_messages = [a[1] for a in annotated if a[-1] == '-1']
    negative_messages = [a[1] for a in annotated if a[-1] == '1']
    neutral_messages = [a[1] for a in annotated if a[-1] == '0']
    non_neg_num = (len(positive_messages) + len(neutral_messages))*ratio
    neg_num = len(negative_messages)*ratio
    print('Non-Negative: {}'.format(non_neg_num), 'Negative: {}'.format(neg_num))
    print(non_neg_num+neg_num)
    return annotated, ratio

def get_messages(lang, messages):
    print("Getting messages")
    text_numbers = [a[1] for a in messages]
    f = open('/home/denis/Documents/aspir/11диссертация/data/' + langs[lang])
    r = f.readlines()
    print("Lines read")
    f.close()
    r = [re.sub('(\t\t).', '\t', l) for l in r]
    r = [re.sub('(\t\n).', '\n', l) for l in r]
    r = [l.strip() for l in r]
    print("Ammount of messages", len(r))
    r = [l.split('\t') for l in r]
    if lang != 'mdk':
        r = [x for x in r if len(x)==6]
    else:
        pass
    wordforms = set([w for m in r for w in m])
    print("Ammount of wordforms", len(wordforms))
    print('Preprocessing finished')
    texts = pd.DataFrame(r)
    print('To dataframe')
    if lang != 'mdk':
        texts = texts.loc[texts[texts.columns[3]].isin(text_numbers)]
        texts = [[x[1],x[3]] for x in texts.values]
    else:
        texts = texts.loc[texts[texts.columns[1]].isin(text_numbers)]
        texts = [[x[3],x[1]] for x in texts.values]
    return(texts)

In [3]:
lang = 'mdk'
annotated, ratio = analyse(lang)
texts_orig = get_messages(lang, annotated)

de
Annotated length: 655
3.0534351145
('Non-Negative: 1624.42748092', 'Negative: 372.519083969')
1996.94656489
Getting messages
Lines read
('Ammount of messages', 667675)
('Ammount of wordforms', 1688133)
Preprocessing finished
To dataframe


In [4]:
annotated2 = {a[1]:[a[2],''] for a in annotated}
my_stopwords = stopwords.words('russian')
my_stopwords = [unicode(w) for w in my_stopwords]
my_stopwords = my_stopwords + stopwords.words('english') + stopwords.words('german') +\
                stopwords.words('spanish')
texts = [[clean_str(t[0]).split(), t[1]] for t in texts_orig]
print(' '.join(texts[0][0]))
for i in range(len(texts)):
    texts[i][0] = [w for w in texts[i][0] if w not in my_stopwords]
for el in texts:
    annotated2[el[1]][-1] = el[0]
print(annotated2)

das layout der seite so zu verschlimmbessern dass kc ohne puppenkanal unbenutzbar ist würde schon reichen des neukrebses fernzuhalten
{'11338418': ['0', [u'pfand', u'sammeln']], '11930998': ['1', [u'sechste', u'fr\xfchst\xfcck', u'lud', u'themaprojektarbeit', u'sozialen', u'bereich', u'p\xe4dagogin', u'mal', u'anderseiniges', u'sozialarbeiterin', u'kira', u'powalowski', u'berufswahl', u'projektleiterin', u'anerkannten', u'bildungstr\xe4ger', u'berufliche', u'reintegration', u'projektverantwortung', u'leitet', u'erfolgreich', u'team', u'ermittelt', u'kreativ', u'weitere', u'm\xf6glichkeiten', u'projektarbeit', u'28', u'diskutierten', u'12', u'frauen', u'chancen', u'geisteswissenschaftlichen', u'studium', u'erfolgreiche', u'projektarbeit', u'sowie', u'abwechslungsreichen', u'aufgaben', u'f\xfchrungsperson', u'schwester', u'genauso', u'nutzloses', u'st\xfcck', u'schei\xdfe', u'bruder', u'labt', u'sozialsystemen', u'wer', u'h\xe4tte', u'gedacht']], '11519896': ['1', [u'hasse', u'coop', u'r

In [5]:
negative = [annotated2[el][1] for el in annotated2 if annotated2[el][0] == '1']
non_negative = [annotated2[el][1] for el in annotated2 if annotated2[el][0] != '1']

In [6]:
print(type(negative[0]))
print(' '.join(negative[0]) + '\n')
print(' '.join(non_negative[0]))

<type 'list'>
sechste frühstück lud themaprojektarbeit sozialen bereich pädagogin mal anderseiniges sozialarbeiterin kira powalowski berufswahl projektleiterin anerkannten bildungsträger berufliche reintegration projektverantwortung leitet erfolgreich team ermittelt kreativ weitere möglichkeiten projektarbeit 28 diskutierten 12 frauen chancen geisteswissenschaftlichen studium erfolgreiche projektarbeit sowie abwechslungsreichen aufgaben führungsperson schwester genauso nutzloses stück scheiße bruder labt sozialsystemen wer hätte gedacht

pfand sammeln


In [7]:
f = open('/home/denis/Documents/aspir/main_code/opencorpora/unigrams.cyr.lc')
o_corpora_unigr = f.readlines()
f.close()
o_corpora_unigr = [l.split()[0].decode('utf-8') for l in o_corpora_unigr]
o_corpora_unigr = set(o_corpora_unigr)

In [8]:
print(' '.join([w[0] for w in FreqDist([w for l in negative for w in l]).most_common() if w[0] not in obscene_voc and w[0] not in o_corpora_unigr and not w[0].startswith(obscene_roots)]))
negative_txt = [w for m in negative for w in m]
non_negative_txt = [w for m in non_negative for w in m]
negative_obscene = []
for m in negative:
    if any([w for w in m if w in obscene_voc or w.startswith(obscene_roots)]):
        negative_obscene.append(m)
non_negative_obscene = []
for m in non_negative:
    if any([w for w in m if w in obscene_voc or w.startswith(obscene_roots)]):
        non_negative_obscene.append(m)
print("Negative messages", len(negative)*ratio)
print('Negative length ', len(negative_txt) * ratio)
print('Negative obscene messages ', len(negative_obscene) * ratio)
print('Negative non-obscene messages ', (len(negative) - len(negative_obscene)) * ratio)
print('Negative obscene ', len([w for w in negative_txt if w in obscene_voc or w.startswith(obscene_roots) or w.endswith(obscene_roots)]) * ratio)
print("Non-Negative messages", len(non_negative)*ratio)
print('Non-Negative length ', len(non_negative_txt) * ratio)
print('Non-Negative obscene messages ', len(non_negative_obscene) * ratio)
print('Non-Negative non-obscene messages ', (len(non_negative) - len(non_negative_obscene)) * ratio)
print('Non-Negative obscene ', len([w for w in non_negative_txt if w in obscene_voc or w.startswith(obscene_roots)]) * ratio)

bernd dass mal schon immer einfach ja nie frauen ach gibt bild krebs wäre mann tun hoffe wer gut sieht vergessen sogar wichser kanaken warum echt hätte säge gab kommen geschichte genau film gar hiv müsste jemals gerade doku zeit würdest bachelorarbeit gmbh punkt menschen hast tust voll gesicht bestimmt konnte zimmer sicher notiz wort meer wirklich tochter winter afd mehr frau eher letzten denken musel falsch alter deutschland djoko nein lüfter number bernds folge intention jahren geht dagen später mutter jemand mach kc kleine familie verdient stehen rechts irgend gez fallout leben schlaf gell darin schnell sicht projektarbeit hitlers natürlich gas heute konservative iq recht nützlich rolle freie gezogen verkaufen dabei genauso meinst finden wek frage sitzen gleich hund leute irgendwie tisch wahrheit vielleicht wärst neidisch daran stecken app bzw problem eigentlich besser dran wochen plätze detektiert wertneutrale gesehen hamburg glutenverzicht aufstubiert metteklintonlüfter4 gf schatt